### Process HYCOM netCDF files to save full (2011-2015) time series of salinity and temperature at interpolated locations of selected hurricane events

In [1]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
from datetime import date, timedelta, datetime
# from datetime import datetime as dtime
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import os.path
import webbrowser
import time
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import matplotlib as mpl
from pathlib import Path


In [2]:
# Box for Philippines and West Pacific
# lat_max = 30
# lat_min = 0
# lon_max = 160
# lon_min = 120
# folder_tag = 'WPacific'

# Box for Baja California - East Pacific
#lat_max = 30
#lat_min = 0
#lon_max = -100
#lon_min = -150
#folder_tag = 'California'

# Box for Florida - West Atlantic
#lat_max = 30
#lat_min = 20
#lon_max = -55
#lon_min = -80
#folder_tag = 'Florida'


############################################################################################################### 
# NEW BOXES (after deciding that doing the the global analysis would have been impossible for time and GBs)

# Box for North Atlantic Ocean 
# lat_max = 50
# lat_min = 20
# lon_max = -10
# lon_min = -80
# folder_tag = 'NorthAtlantic'

############################# WEST PACIFIC (3 boxes)
# Box for West Pacific 1 of 3
lat_max = 30
lat_min = 0
lon_max = 160
lon_min = 120
folder_tag = 'WPacific_1of3'

# Box for West Pacific 2 of 3
# lat_max = 50
# lat_min = 30
# lon_max = 180
# lon_min = 130
# folder_tag = 'WPacific_2of3'

# Box for West Pacific 3 of 3 
# lat_max = 30
# lat_min = 18
# lon_max = 180
# lon_min = 160
# folder_tag = 'WPacific_3of3'

############################# SOUTH WEST PACIFIC (2 boxes)
# Box for South West Pacific 1 of 2
# lat_max = -8
# lat_min = -32
# lon_max = 180
# lon_min = 145
# folder_tag = 'SWPacific_1of2'

# Box for South West Pacific 2 of 2 
# lat_max = -10
# lat_min = -32
# lon_max = -170
# lon_min = -180
# folder_tag = 'SWPacific_2of2'

############################ EAST PACIFIC
# Box for East Pacific Ocean
# lat_max = 30
# lat_min = 0
# lon_max = -100
# lon_min = -150
# folder_tag = 'EPacific'

################################# INDIAN OCEAN
# Box for Indian Ocean 
# lat_max = -10
# lat_min = -30
# lon_max = 125
# lon_min = 45
# folder_tag = 'Indian'

############################################################################################################### 


HYCOM_raw_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/HYCOM_near_TC_53x_raw_box_NEW_BOX_' + folder_tag + '/'
HYCOM_save_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/5_HYCOM_near_TC_53x_pkl_NEW_BOX_' + folder_tag + '/'

# HYCOM_raw_dir = '/Volumes/My Passport for Mac/MAC_15"/HYCOM/HYCOM_RAW_DATA_8_boxes/HYCOM_near_TC_53x_raw_box_NEW_BOX_' + folder_tag + '/'

tag_file = 'HYCOM_53X'
max_depth_index_sel = 28 #28 
var_names_download = ['water_temp', 'salinity']
var_names = ['temperature', 'salinity']
file_extension = '.nc4'



In [4]:
# Create list of time-dates of HYCOM files that need to be opened

start = datetime(2011, 1, 1, 00, 00, 00)
# start = datetime(2014, 11, 28, 00, 00, 00)
end = datetime(2015, 12, 31, 21, 00, 00)
date_generated = [start + timedelta(hours=3*x) for x in range(0, (end-start).days*8)]

# date_generated

### Open forcing file with selected hurricane events, with WS and precipitation data


In [5]:
# Open .csv file
df_forcing = pd.read_csv('df_forcing_hur_events_NEW_BOX_with_functions_' + str(lat_min) + '_' + str(lat_max) + '_' + str(lon_min) + '_' + str(lon_max) + '.csv')
# Convert lon from [0,360] to [-180,180]
#df_forcing.LONG = (df_forcing.LONG + 180) % 360 - 180


In [6]:
df_forcing

,Unnamed: 0,LAT,LONG,DATETIME,WIND,PRECIPITATION,ID,level_0,index
0,0,15.229980,128.249999,2012-09-24 20:00:00,62.852540,0.002621,1,60806,266926
1,0,15.025551,134.386363,2014-08-03 03:00:00,62.774414,0.019100,2,158150,622346
2,0,11.345824,125.795454,2013-11-07 23:00:00,61.185547,0.002134,3,137717,506480
3,0,9.914820,130.909090,2013-11-07 09:00:00,60.543945,0.015859,4,135787,504465
4,0,16.660985,131.727272,2014-08-03 21:00:00,60.310547,0.000777,5,161023,625219
...,...,...,...,...,...,...,...,...,...
437,0,18.296419,120.068181,2011-09-27 05:00:00,33.038086,0.000000,438,29152,149402
438,0,11.754683,143.795454,2013-10-17 13:00:00,32.996094,0.002797,439,117909,484133
439,0,17.274273,128.659090,2012-09-25 08:00:00,32.946290,0.004267,440,62474,268603
440,0,19.931854,124.772726,2014-07-22 02:00:00,32.939453,0.008952,441,153839,615870


### For each event, loop over time and process and save full time series of HYCOM data at that location

In [33]:
for x in date_generated: # loop through all unique times
    print(x)
    
    for i in np.arange(df_forcing.shape[0]):
        # Latitude and longitude of event being considered
        sel_lat = df_forcing.iloc[i].LAT
        sel_lon = df_forcing.iloc[i].LONG
    
        for namevar_download,namevar in zip(var_names_download,var_names): # loop over variables (temp or salt)
            # Open HYCOM file for this date-time
            file_prefix ='?var=VARIABLE&north=NORTH&west=WEST&east=EAST&south=SOUTH&horizStride=1&time='
            file_suffix='Thh%3A00%3A00Z&vertCoord=&addLatLon=true&accept=netcdf4'
            file_prefix = file_prefix.replace('VARIABLE', str(namevar_download)) 
            north = str(max(np.ceil(lat_max), np.floor(lat_max)))
            south = str(min(np.ceil(lat_min), np.floor(lat_min)))
            east = str(max(np.ceil(lon_max), np.floor(lon_max)))
            west = str(min(np.ceil(lon_min), np.floor(lon_min)))
            date_to_download = pd.to_datetime(x)
            year = str(date_to_download)[0:4]
            month = str(date_to_download)[5:7]
            day = str(date_to_download)[8:10]
            hour = str(date_to_download)[11:13]
            date = datetime(int(year), int(month), int(day), int(hour))
            year = f"{int(date_to_download.year):04d}"
            month = f"{int(date_to_download.month):02d}"
            day = f"{int(date_to_download.day):02d}"
            file_suffix_hh = file_suffix.replace('hh', hour, 1)
            file_prefix_coord = file_prefix.replace('NORTH', north, 1)
            file_prefix_coord = file_prefix_coord.replace('SOUTH', south, 1)
            file_prefix_coord = file_prefix_coord.replace('EAST', east, 1)
            file_prefix_coord = file_prefix_coord.replace('WEST', west, 1)

            my_file = Path(HYCOM_raw_dir + f'{file_prefix_coord}{year}{month}{day}{file_suffix_hh}{file_extension}')
            if my_file.is_file():
                bfr_dset = xr.open_dataset(my_file)
                # Longitude is -180 + 180 already, so no need to re-project
                STOOOOP
                # Interpolate HYCOM data and keep top max_depth_index_sel levels
                if namevar == 'temperature':
                    temperature = bfr_dset.interp(lon=[sel_lon], lat=[sel_lat])[namevar_download].values.flatten().tolist()[0:max_depth_index_sel]
                elif namevar == 'salinity':
                    salinity = bfr_dset.interp(lon=[sel_lon], lat=[sel_lat])[namevar_download].values.flatten().tolist()[0:max_depth_index_sel]

        # Save 3-hourly files with HYCOM data near TCs
        index = np.arange(0,bfr_dset.time.shape[0],1)

        d_xr = xr.Dataset(
            data_vars=dict(
                longitude=(["index"], [sel_lon]),
                latitude=(["index"], [sel_lat]),
                time=(["index"], bfr_dset.time.values), # add a check: il time step è lo stesso del nome del file?
                #wind=(["index"], df_on_TC_track.WIND), # FROM FORCING ONCE DOWNLOADED
                #precipitation=(["index"], df_on_TC_track.PRECIPITATION), # FROM FORCING ONCE DOWNLOADED
                temperature=(["index","depth"], [temperature] ,{'label': 'HYCOM T at TC location'}),
                salinity=(["index","depth"], [salinity],{'label': 'HYCOM S at TC location'}),
            ),
            coords=dict(
                index=index,
                depth=bfr_dset.depth.values[0:max_depth_index_sel],
            ),
            attrs=dict(description="HYCOM T/S co-located with TCs"),
            )
        with open(HYCOM_save_dir+tag_file+'_' + str(sel_lat) + '_' + str(sel_lon) + '_' + \
                  str(x.year)+'_'+"{:02d}".format(x.month)+'_'+"{:02d}".format(x.day)+'_'+"{:02d}".format(x.hour)+'UTC.pkl', 'wb') as handle:
            pickle.dump(d_xr, handle, protocol=pickle.HIGHEST_PROTOCOL)      


2011-01-01 00:00:00


NameError: name 'STOOOOP' is not defined